# Исследование объявлений о продаже квартир

Целью настоящего исследования является изучение рынка недвижимости и его динамики в Санкт-Петербурге и Ленинградской области. Для этого на основе имеющихся данных необходимо выявить факторы, оказывающие наиболее значительное влияние на стоимость квартир. Полученные выводы могут быть в дальнейшем использованы бизнесом для автоматизации поиска и выявления аномалий и мошенничества.
В процессе своего исследования я планирую осуществить следующие шаги:
1. Открыть файл с данными, изучить информацию о них, оценить возможные ошибки.
2. Обработать пропуски и ошибки, где это представится возможным для минимизации их влияния на результат исследования.
3. Добавить в таблицу новые данные на основе произведенных рассчетов, в некоторых столбцах - изменить тип данных, чтобы облегчить работу с ними, а также использовать их для дальнейшего анализа с максимальной пользой.
4. Произвести анализ зависимости целевой переменной (стоимости объектов) от указанных в таблице параметров, визуализировать полученные выводы для наглядности.
5. Оценить рыночную стоимость квартир для Санкт-Петербурга и населенных пунктов в области, выявить отличия в цене для разных регионов, а также на различном удалении от центра города.
6. Суммировать полученную информацию и выявленные закономерности, а также выдвинуть гипотезы в общем выводе.

### Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd

In [3]:
data = pd.read_csv('/datasets/real_estate_data.csv', sep = '\t')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1729477489.py, line 1)

In [ ]:
data.info()

In [ ]:
data.hist(figsize=(15, 20));

In [ ]:
rows_quantity = data.shape[0]
rows_quantity

На этапе изучения информации о данных был загружен датасет с данными, состоящими из 23 699 объектов и информации о них (местоположении, основных площадях, размерах и этажности, дате публикации объявления, ну и, конечно, стоимости). Было изучено количество и тип данных в каждом столбце, а также построены гистограммы, дающие наглядное представление об объеме данных и их примерном количественном распределении в каждом параметре. 

### Выполните предобработку данных

In [ ]:
data.isna().sum()


In [ ]:
data['balcony'] = data['balcony'].fillna(value = 0)
for i in data['locality_name'].unique():
    data.loc[(data['locality_name'] == i) & (data['airports_nearest'].isna()), 'airports_nearest'] = data.loc[(data['locality_name'] == i), 'airports_nearest'].mean()
    data.loc[(data['locality_name'] == i) & (data['parks_around3000'].isna()), 'parks_around3000'] = data.loc[(data['locality_name'] == i), 'parks_around3000'].mean()
    data.loc[(data['locality_name'] == i) & (data['ponds_around3000'].isna()), 'ponds_around3000'] = data.loc[(data['locality_name'] == i), 'ponds_around3000'].mean()
    data.loc[(data['locality_name'] == i) & (data['parks_nearest'].isna()), 'parks_nearest'] = data.loc[(data['locality_name'] == i), 'parks_nearest'].mean()
    data.loc[(data['locality_name'] == i) & (data['ponds_nearest'].isna()), 'ponds_nearest'] = data.loc[(data['locality_name'] == i), 'ponds_nearest'].mean()
    data.loc[(data['locality_name'] == i) & (data['cityCenters_nearest'].isna()), 'cityCenters_nearest'] = data.loc[(data['locality_name'] == i), 'cityCenters_nearest'].mean()


In [ ]:
#итоговое количество пропусков после обработки
data.isna().sum()

Пропущенные значения были обработаны следующим образом:
    
   1. В столбцах с информацией о наличии балконов пропуски были заменены на ноль. Можно предположить, что данная информация была получена от пользователей, которые в пункте про наличие балкона в квартире могли оставить пропуск, если балкона нет.
   2. Пропуски в остальных столбцах, судя по всему, тоже заполненных пользователями, объясняются недостаточностью информации и человеческими ошибками. Самое большое количество пропусков, к примеру, в столбце с информацией о том, является ли объект апартаментами. Можно предположить, что пользователь просто не владеет данной информацией или считает ее непринципиальной для продажи жилья. Пропуски в данном столбце оставлены без изменений, как и в других столбцах, заполненных пользователями. Что можно было сделать: заполнить жилую площадь и площадь кухни исходя из среднего отношения их величин к общей площади квартиры.
   3. Следующая группа данных с пропусками была восполнена в соответствии с  информацией о местоположении объекта. Судя по всему, эта группа данных получена автоматически на основе картографических данных, и в этом случае пропуски скорее всего появились по техническим причинам, связанным с ошибками при переносе и копировании данных, а также ошибками программистов. Заполнение было произведено на основании среднего числа для объектов, находящихся в том же населенном пункте (не медиана, так как для некоторых районов представлен только один объект). Таким образом была заполнена значительная часть пропущенных значений из столбцов расстояние до центра города и до ближайшего водоема и не очень большая часть пропусков из столбцов Расстояние до аэропорта, Число парков и прудов в радиусе 3 км, а также Расстояние до ближайшего парка.

In [ ]:
#Обработка дубликатов в названиях населенных пунктов
data['locality_name'] = (
    data['locality_name']
    .str.replace('ё', 'е')
    .replace(['городской поселок', 'поселок городского типа', 'садовое товарищество', 'садоводческое некоммерческое товарищество', 'коттеджный поселок', 'деревня', 'поселок станции', 'поселок при железнодорожной станции'], 'поселок', regex=True)
    .replace(['село '], 'поселок ', regex=True)
)


   
print(data['locality_name'].unique())

In [ ]:
#преобразуем тип данных столбцов
data['first_day_exposition'] = pd.to_datetime(data['first_day_exposition'], format = '%Y-%m-%dT%H:%M:%S')
data['balcony'] = data['balcony'].astype(int)
data['last_price'] = data['last_price'].astype(int)

In [ ]:
#Проверка на явные дубликаты
print(data.duplicated().sum())

Выполнено преобразование типов данных в следующих столбцах:

   1. Дата публикации объявления - данные из столбца преобразованы в формат DateTime для упрощения работы с датами.
   2. Столбец с количеством балконов в квартире был преобразован в целочисленный тип, что стало возможным после удаления пропусков.
   3. Тип данных в столбце со стоимостью объекта был преобразован в целочисленный как наиболее подходящий для работы с ценами на недвижимость.

### Добавьте в таблицу новые столбцы

In [ ]:
#Добавление столбца в таблицу со стоимостью квадратного метра жилья
data['square_meter_price'] = round(data['last_price'] / data['living_area'], 2)

display(data.head(20))

In [ ]:
#Добавление столбцов с днем недели, месяцем и годом появления объявления
data['exposition_weekday'] = data['first_day_exposition'].dt.weekday
data['exposition_month'] = data['first_day_exposition'].dt.month
data['exposition_year'] = data['first_day_exposition'].dt.year
display(data.head(20))

In [ ]:
#тип этажа квартиры (значения — «первый», «последний», «другой»)
def floor_category(row):
    
    floor = row['floor']
    if floor == row['floors_total']:
        return 'Последний'
    if floor == 1:
        return 'Первый'
    return 'Другой'

data['floor_type'] = data.apply(floor_category, axis = 1)
display(data.head(20))
print(data['floor_type'].unique())

In [ ]:
#расстояние до центра города в километрах
data['to_city_center'] = round(data['cityCenters_nearest'] / 1000)
display(data.head(20))

### Проведите исследовательский анализ данных

Данные в значимых для исследования столбцах необходимо оценить на предмет разброса значений и выявить аномалии. По возможности восстановить корректное значение либо удалить данные с аномальными значениями.
Первый столбец данных для анализа - общая площадь. Разброс данных тут очень большой, но основная часть находится в диапазоне от 40 до 69 квадратных метров. Трудно оценить предел аномальности значений и тем более их скорректировать. Что было сделано: удалены строки со значением общей площади более 350 квадратных метров, которых было очень мало (41 строка) и их удаление не влияет на статистику (как видно из переменной filtered_rows, после удаления осталось больше 99% данных).
Остальные данные, сильно отличающиеся от средней квартили, были оставлены неизмененными, исходя из предположения, что в жилищном фонде вполне могут присутствовать объекты с большой площадью.
Что можно было бы сделать: скорректировать общую площадь, исходя из среднего значения для категории объектов по количеству комнат.

In [ ]:
data.hist('total_area', range = (0, 400))
data = data.loc[data['total_area'] < 350 ]
print(data['total_area'].describe())
filtered_rows = data.shape[0]
print(filtered_rows/rows_quantity)

С жилой площадью дело обстоит похожим образом. При ближайшем рассмотрении данного столбца можно отследить, что основная часть данных находится в диапазоне 18 - 42 квадратных метра. Условной границей отсечения аномалий выбрано значение в 170 кв.м., так как объектов с большей жилой площадью менее 1% от общего количества (45 объектов).

In [ ]:
#Жилая площадь
data.hist('living_area', range = (0, 250))
print(data['living_area'].describe())
data = data.loc[(data['living_area'] < 170) | (data['living_area'].isna())]

filtered_rows2 = data.shape[0]
print(filtered_rows2/rows_quantity)

Основной разброс значений в площади кухни находится в диапазоне от 7 до 12 квадратных метров, но есть и аномалии с маленькой площадью (не нулевой) и большой (до 72 кв.м). Было решено отбросить объекты со значениями площади кухни более 90 кв.м (их всего 4), при этом не отбрасывая значения с пропусками, в целях большей однородности данных для проведения анализа.

In [ ]:
# Площадь кухни
data.hist('kitchen_area')
print(data['kitchen_area'].describe())
data = data.loc[(data['kitchen_area'] < 90) | (data['kitchen_area'].isna())]
print(data.shape[0]/rows_quantity)

Цена объекта может зависеть от огромного количества факторов, указанных в изначальной таблице с данными. Огромный разброс и наличие аномалий в большом количестве столбцов, а также разные населенные пункты и другие критерии не могут не влиять на большую разницу минимальной и максимальной стоимости объекта. Однако, из данных удалены 17 объектов, стоимость которых была точечно разбросана в верхнем максимуме. Их наличие оказывало значительно влияние на выявление зависимостей общей стоимости от различных факторов. После применения фильтра осталось около 99% данных.

In [ ]:
#Цена объекта
data.hist('last_price', range = (0, 4.700000e+07))
print(data['last_price'].describe())
data = data.loc[data['last_price'] < 9.300000e+07]
print(data.shape[0]/rows_quantity)

Количество комнат начинается от одной и доходит до 11. Если принимать во внимание самые большие по площади объекты, этот признак не выбивается из общей картины.

In [ ]:
#Количество комнат
data.hist('rooms')
print(data['rooms'].describe())

Высота потолков более 20 метров была обработана исходя из предположения, что, заполняя эту информацию, пользователь мог ошибиться с десятичной запятой. Данные были скорректированы.

In [ ]:
#Высота потолков
data.hist('ceiling_height', range = (0, 8.5))
print(data['ceiling_height'].describe())

data['ceiling_height'] = data['ceiling_height'].apply(lambda x: x/10 if x>=20 else x)


Для визуализации данных о типах этажей объектов была составлена сводная таблица для каждой категории, а затем построен столбчатый график. Как видно из приведенного ниже анализа, большее количество объектов приходится на "Другой" (то есть не первый и не последний этаж), далее идут объекты на последнем этаже, а меньше всего объектов находятся на первом.

In [ ]:
#тип этажа квартиры

floor_type = data.pivot_table(index = 'floor_type', values = 'days_exposition', aggfunc = 'count')
floor_type.columns = ['Количество']
display(floor_type)
floor_type.plot(kind = 'bar')

Были удалены объекты с общим количеством этажей в доме более 37, так как я проверила, что в Санкт-Петербурге нет жилых домов большей этажности. После проведения последней фильтрации осталось более 99% данных.


In [ ]:
#общее количество этажей в доме
data.hist('floors_total')
data = data.loc[(data['floors_total'] < 37) | (data['floors_total'].isna())]
print(data['floors_total'].describe())
print(data.shape[0]/rows_quantity)


Разброс расстояния от центра города (Санкт-Петербург) объясняется большим количеством населенных пунктов в данных, для минимального это центр города, максимальное расстояние тоже вполне вписывается в местоположение объектов.

In [ ]:
#расстояние до центра города в метрах
data.hist('cityCenters_nearest')
print(data['cityCenters_nearest'].describe())

Расстояние до ближайшего парка, судя по всему, как и другие данные о расстояниях, получены автоматически из геосервисов. Сильно отличающиеся от средних данные оставлены без изменений.

In [ ]:
#расстояние до ближайшего парка
data.hist('parks_nearest')
data['parks_nearest'].describe()

In [ ]:
#Изучите, как быстро продавались квартиры
data.hist('days_exposition', bins = 100, range = (0, 200))
print('Среднее количество дней, которое были размещены объявления:', data['days_exposition'].mean())
print('Медианное количество дней, которое были размещены объявления:', data['days_exposition'].median())
print(data['days_exposition'].describe())

Медианное значение, по которому можно судить о стандартном времени продажи объекта, - 102 дня. Среднее значение превышает данный показатель, значит, некоторые объекты продаются сильно дольше остальных. Быстрыми можно считать продажи квартир, входящих в первую квартиль, то есть до 44 дней. Как видно из гистограммы при ближайшем рассмотрении, количество продаваемых квартир снижается со временем, постепенно выходя на плато после 100 дней. Есть несколько всплесков количества проданных объектов в следующие промежутки времени: самый явный около 45 дней после размещения объявления (количество квартир более 1000), меньший примерно на отметке в 60 дней и еще меньший предположительно через 80-85 дней. 
Необычно долгими можно считать сроки продажи более 400 - 500 дней (они не вошли в масштаб графика).

In [ ]:
#Факторы, которые больше всего влияют на общую (полную) стоимость объекта

print(data['last_price'].corr(data['total_area']))
total_area_price_data = data.pivot_table(index = 'total_area', values = 'last_price')
display(total_area_price_data)

data.plot(x = 'last_price', y = 'total_area', kind = 'scatter', title = 'Зависимость стоимости от общей площади')

Как и ожидалось, общая стоимость объекта зависит от общей площади: при ее повышении повышается и цена. Как видно из графика и рассчитанного коэффициента корреляции, зависимость достаточно сильная.

In [ ]:
print(data['last_price'].corr(data['living_area']))
living_area_price_data = data.pivot_table(index = 'living_area', values = 'last_price')
display(living_area_price_data)

data.plot(x = 'last_price', y = 'living_area', kind = 'scatter', title = 'Зависимость стоимости от жилой площади')



Размер жилой площади тоже оказывает влияние на стоимость объекта, но уже в меньшей степени, чем общая площадь (коэффициенты корреляции 0,65 и 0,75 соответственно).

In [ ]:
print(data['last_price'].corr(data['kitchen_area']))
kitchen_area_price_data = data.pivot_table(index = 'kitchen_area', values = 'last_price')
display(kitchen_area_price_data)

data.plot(x = 'last_price', y = 'kitchen_area', kind = 'scatter', title = 'Зависимость стоимости от площади кухни')



Выявлена прямая корреляция стоимости квартиры и площади кухни (отражена на графике рассеяния). Коэффициент корреляции еще ниже, чем у жилой площади, следовательно, зависимость еще более слабая.

In [ ]:
print(data['last_price'].corr(data['rooms']))
rooms_price_data = data.pivot_table(index = 'rooms', values = 'last_price')
display(rooms_price_data)

rooms_price_data.plot(kind = 'bar', title = 'Зависимость стоимости от количества комнат')


Зависимость стоимости объекта от количества комнат тоже присутствует, что видно на графике и по коэффициенту корреляции. Хотя можно отследить повышение стоимости квартир от 0 до 6 комнат, далее разница незначительна, а для квартир с 10 комнатами средняя цена даже ниже. Стоимость квартир с большим количеством комнат (11 и более) наиболее высокая на графике.

In [ ]:
#тип этажа
floor_price = data.pivot_table(index = 'floor_type', values = 'last_price')
display(floor_price)
floor_price.plot(kind = 'bar', title = 'Стоимость в зависимости от категории этажа')

На графике видно, что квартиры на первом этаже стоят в среднем значительно дешевле, чем на других и последнем этажах, а вот разница между последними двумя категориями незначительна и может быть объяснена неоднородностью выборок (квартир на других этажах значительно больше).

In [ ]:
#день недели публикации объявления
weekday_price = data.pivot_table(index = 'exposition_weekday', values = 'last_price')
display(weekday_price)
weekday_price.plot(color = 'g', title = 'Динамика стоимости объектов по дням недели')

На графике видно, что в объявлениях, опубликованных в субботу, указана наиболее низкая средняя стоимость квартир, а в среду наиболее высокая. Данная гипотеза требует проверки.

In [ ]:
#месяц публикации объявления
month_price = data.pivot_table(index = 'exposition_month', values = 'last_price')
display(month_price)
month_price.plot(title = 'Динамика стоимости объектов по месяцам')

Судя по приведенному ниже графику, средняя стоимость квартир в объявлениях, опубликованных в октябре, наиболее низкая, а в последние два месяца сильно повышается к своему максимуму в декабре. Январь также характеризуется достаточно высокими значениями. То есть пик цен приходится на конец и начало года. Причины такой неоднородности в данных требуют дальнейшего исследования.

In [ ]:
#год публикации объявления
year_price = data.pivot_table(index = 'exposition_year', values = 'last_price', aggfunc = 'median')
display(year_price)
area_price = data.pivot_table(index = 'exposition_year', values = 'total_area', aggfunc = 'median')
display(area_price)
year_price.plot(color = 'y', title = 'Динамика цены с 2014 по 2019 год')
area_price.plot(color = 'b', title = ' Динамика площади продаваемых квартир')

Видно, что не только средняя стоимость квартир, но и общая площадь продаваемых объектов с 2014 до 2018 года упали, причем за год достаточно резко, потом снижение происходило более плавно. Затем к 2019 году оба этих показателя пошли на плавное повышение. Данная гипотеза предоставляет простор для дальнейших исследований рынка и изменения цен на недвижимость.

In [ ]:
#Средняя цена одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений
largest_number = data.pivot_table(index = 'locality_name', values = 'square_meter_price', aggfunc = ['count', 'median']).sort_values(by = ('count', 'square_meter_price'), ascending = False).head(10)
largest_number.columns = ['Количество объявлений', 'Медианная цена квадратного метра']
display(largest_number.sort_values(by = 'Медианная цена квадратного метра', ascending = False))


В таблице выше приведены населенные пункты с наибольшим количеством объявлений в порядке убывания цены за квадратный метр. Как и ожидалось, самая высокая средняя стоимость в Санкт-Петербурге, а вот медианная выше в поселке Парголово.  На последнем месте - город Выборг.

In [ ]:
#Квартиры в Санкт-Петербурге со средней стоимостью на разном удалении от центра
spb_objects = data[data['locality_name'] == 'Санкт-Петербург'].pivot_table(index = 'to_city_center', values = 'last_price', aggfunc = 'mean')
display(spb_objects.sort_values(by = 'to_city_center'))
spb_objects.plot(color = 'r', title = 'Стоимость квартир по мере удаления от центра')

В таблице приведены данные о средней стоимости квартир в Санкт-Петербурге по мере удаления от центра города. График демонстрирует обратную зависимость: чем ближе к центру находится объект, тем стоимость его выше. Небольшие всплески стоимости также находятся на расстоянии примерно 6 км и 26-27 км от центра города, что может быть объяснено другими факторами (благополучным районом с развитой инфраструктурой или современным жилым комплексом более высокого класса на некотором удалении от центра).

### Напишите общий вывод

Было проведено исследование массива из 23699 строк данных о продаже квартир в Санкт-Петербурге и его окрестностях (до 60 км), которые были получены из двух источников: заполнены пользователями лично и автоматически сформированы на основе картографических данных. Целью данного исследования было выявления аномалий в целях предотвращения мошенничества, а также выявление факторов, оказывающих влияние на итоговую стоимость объекта.

На начальном этапе был изучен тип данных в каждом столбце и их объемы через построение гистограмм для каждого столбца.
На этапе предобработки данных были проведены следующие этапы:

   1. Обработка пропусков (исправление на 0 в случае с количеством балконов, не указанных пользователем при их отсутствии, и заполнение средним значением на основе картографических данных о местоположении).
   2. Изменение типов данных в столбцах, где возможно, для облегчения работы с ними (тип даты и целочисленный тип).
   3. Добавление новых столбцов с полученными данными: день недели, месяц и год публикации объявления, стоимость квадратного метра жилья, расстояние до центра города в километрах, тип этажа, на котором находится объект. Новые данные в дальнейшем будут использованы для проведения анализа.
   4. Обработка дубликатов в названиях населенных пунктов путем унификации названий.
    
На этапе исследования была проведена обработка аномалий. Путем построения количественных гистограмм выявлены аномальные значения, оценена их доля в общем массиве данных. Крайние значения были отфильтрованы с соблюдением условия их небольшой доли. После отсечения аномалий зависимости стали прослеживаться отчетливее. Аномалии в высоте потолков были обработаны особым образом: добавлена десятичная запятая исходя из предположения об ошибках заполнения.

Затем последовал этап выявления зависимостей стоимости квартиры от различных параметров. Были построены графики зависимостей, а также посчитаны коэффициенты корреляции, где это возможно. Были выявлены зависимости (по мере уменьшения коэффициента) от общей площади, жилой площади и размера кухни. Зависимость от количества комнат наблюдается для квартир с количеством комнат от 1 до 3, далее она ослабевает. Также была выявлена более низкая стоимость для объектов, расположенных на первом этаже, по сравнению с другими. Были выдвинуты гипотезы касательно зависимости стоимости квартиры от дня недели, месяца и года публикации объявления. Они требуют дальнейших проверок и исследований:

   1. В среду - более высокая стоимость, в субботу - более низкая.
   
   2. Пик стоимости объектов приходится на объявления конца и начала года, а в октябре эта стоимость минимальная.

   3. С 2014 до 2018 года наблюдается площади продаваемых объектов, что приводит и к снижению их стоимости. К 2019 году начинается плавное повышение обоих параметров.

На заключительном этапе были выявлены десять населенных пунктов с наибольшим количеством объявлений, из которых выделены пункты с наиболее высокой (ожидаемо Санкт-Петербург) и низкой (Выборг) стоимостью квадратного метра. Также было проанализировано изменение общей стоимости квартир по мере удаления от центра города Санкт-Петербург. Стоимость ожидаемо снижалась с небольшими всплесками на удалении в 6 и 26-27 километров.